# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_weather_df=pd.read_csv('city_weather_df.csv').drop('Unnamed: 0',axis=1).dropna()
city_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Dhidhdhoo,100,MV,1569385051,77,6.88,73.10,81.53,21.97
1,Albany,1,US,1569384830,100,42.65,-73.75,59.00,3.36
2,Loudi,100,CN,1569385052,32,27.73,112.00,81.53,3.98
3,Mataura,100,NZ,1569385052,93,-46.19,168.86,41.00,5.01
4,Kangasala,75,FI,1569385053,93,61.46,24.07,41.00,4.70


In [ ]:
counter +=1

print(counter)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
import gmaps

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)


# Plot Heatmap
fig = gmaps.figure()


# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = city_weather_df["Humidity"].astype(float)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=0.8,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
narrowed_city_df=city_weather_df[(city_weather_df['Max Temp']<80)&
                (city_weather_df['Max Temp']>70)&
                (city_weather_df['Wind Speed']<10)&
                (city_weather_df['Cloudiness']==0)]
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,Rikitea,0,PF,1569385055,65,-23.12,-134.97,71.09,3.18
26,Cartagena,0,ES,1569385058,77,37.63,-1.00,71.01,6.93
61,Hohhot,0,CN,1569385067,27,40.81,111.65,73.40,4.47
167,Shimoni,0,KE,1569385091,81,-4.65,39.38,70.55,4.00
193,Urupes,0,BR,1569385098,50,-21.20,-49.29,78.80,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df=narrowed_city_df.copy()
hotel_df['Hotel Name']=""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,Rikitea,0,PF,1569385055,65,-23.12,-134.97,71.09,3.18,
26,Cartagena,0,ES,1569385058,77,37.63,-1.00,71.01,6.93,
61,Hohhot,0,CN,1569385067,27,40.81,111.65,73.40,4.47,
167,Shimoni,0,KE,1569385091,81,-4.65,39.38,70.55,4.00,
193,Urupes,0,BR,1569385098,50,-21.20,-49.29,78.80,6.93,


In [37]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    #"keyword": "international airport",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,Rikitea,0,PF,1569385055,65,-23.12,-134.97,71.09,3.18,Rikitea
26,Cartagena,0,ES,1569385058,77,37.63,-1.00,71.01,6.93,Region of Murcia
61,Hohhot,0,CN,1569385067,27,40.81,111.65,73.40,4.47,Hohhot
167,Shimoni,0,KE,1569385091,81,-4.65,39.38,70.55,4.00,Diani Beach
193,Urupes,0,BR,1569385098,50,-21.20,-49.29,78.80,6.93,São José do Rio Preto


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=0.8,
                                 point_radius=2)

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))